In [215]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np

SentenceBERT 모델 로드

In [216]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ['안녕하세요?', '한국어 문장 임베딩을 위한 버트 모델입니다.']
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510458 -0.7733844   0.5927712  ...  0.5792354   0.32683465
  -0.6508966 ]
 [-0.09361678 -0.18191552 -0.1923082  ... -0.03165798  0.30412537
  -0.26793647]]


데이터셋 로드

웰니스 대화 스크립트 데이터셋

In [10]:
# df = pd.read_csv('wellness_dataset_original.csv')

# df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


In [253]:
df_pet = pd.read_csv('df_pet.csv')

In [254]:
df_pet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37209 entries, 0 to 37208
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  37209 non-null  int64 
 1   대화          37209 non-null  object
 2   구분          37209 non-null  int64 
 3   주제          37209 non-null  object
 4   의도          36989 non-null  object
 5   화자수         37209 non-null  object
 6   대화나누기       37209 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 2.0+ MB


In [255]:
df_pet.head()

,Unnamed: 0,대화,구분,주제,의도,화자수,대화나누기
0,1216,1 : 이구아나나 파충류 키워본 적 있나?,49,반려동물,(지시) 질문하기,1:1,1
1,1217,2 : 아니 무서워서 어떻게 키워…,49,반려동물,(지시) 질문하기,1:1,2
2,1218,1 : 키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,49,반려동물,(지시) 질문하기,1:1,1
3,1219,2 : 며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,49,반려동물,(지시) 질문하기,1:1,2
4,1220,1 : 어디 동물원 갔었엉?,49,반려동물,(지시) 질문하기,1:1,1


In [256]:
df_pet_s = df_pet.loc[:10004,:]

In [237]:
df_pet_s['대화'].str.split(' : ').head()

0                             [1, 이구아나나 파충류 키워본 적 있나?]
1                                 [2, 아니 무서워서 어떻게 키워…]
2                   [1, 키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ]
3    [2, 며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서]
4                                     [1, 어디 동물원 갔었엉?]
Name: 대화, dtype: object

In [257]:
df_pet_s2 = pd.DataFrame(columns=['의도','대화나누기','입력','출력'])

for s in tqdm(range(len(df_pet_s) - 1)):
    df_pet_s2.at[s, '의도'] = df_pet_s.loc[s, '의도']
    df_pet_s2.at[s, '대화나누기'] = df_pet_s.loc[s, '대화나누기']
    current_value = df_pet_s.loc[s, '대화'].split(' : ')
    next_value = df_pet_s.loc[s+1, '대화'].split(' : ')
    if (int(current_value[0]) == int(next_value[0])) & (int(current_value[0]) == 1): 
        df_pet_s2.at[s, '입력'] = current_value[1] + ' ' + next_value[1]
    elif (int(current_value[0]) == int(next_value[0])) & (int(current_value[0]) == 2):
        df_pet_s2.at[s-1, '출력'] = current_value[1] + ' ' + next_value[1]
    elif int(current_value[0]) == 1:
        df_pet_s2.at[s, '입력'] = current_value[1]
    elif int(current_value[0]) == 2:
        df_pet_s2.at[s-1, '출력'] = current_value[1]



100%|██████████| 10004/10004 [00:55<00:00, 180.79it/s]


In [239]:
df_pet_s2.head(50)

,의도,대화나누기,입력,출력
0,(지시) 질문하기,1,이구아나나 파충류 키워본 적 있나?,아니 무서워서 어떻게 키워…
1,(지시) 질문하기,2,NaN,NaN
2,(지시) 질문하기,1,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서
3,(지시) 질문하기,2,NaN,NaN
4,(지시) 질문하기,1,어디 동물원 갔었엉?,제파라고 제주도에 있어 하하
5,(지시) 질문하기,2,NaN,NaN
6,(지시) 질문하기,1,거기 파충류 막 만지고 할 수 있어? 키키,아 키키 체험하는 곳이라서 만질 수 있다
7,(지시) 질문하기,2,NaN,NaN
8,(지시) 질문하기,1,오 키키 좋은 구경했겠다 이구아나 만졌을 때 느낌 어땠어?,NaN
9,(지시) 질문하기,1,이구아나 만졌을 때 느낌 어땠어?,거칠거칠한 느낌? 무서웠다


In [258]:
df_pet_s3 = df_pet_s2[~df_pet_s2['입력'].isnull() & ~df_pet_s2['출력'].isnull()]

In [259]:
df_pet_s3.head(10)

,의도,대화나누기,입력,출력
0,(지시) 질문하기,1,이구아나나 파충류 키워본 적 있나?,아니 무서워서 어떻게 키워…
2,(지시) 질문하기,1,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서
4,(지시) 질문하기,1,어디 동물원 갔었엉?,제파라고 제주도에 있어 하하
6,(지시) 질문하기,1,거기 파충류 막 만지고 할 수 있어? 키키,아 키키 체험하는 곳이라서 만질 수 있다
9,(지시) 질문하기,1,이구아나 만졌을 때 느낌 어땠어?,거칠거칠한 느낌? 무서웠다
12,(지시) 질문하기,1,어우 키키 말만 들어도 무섭농,너는 이구아나 본 적 있어?
14,(지시) 질문하기,1,아니 ㅠㅠ 나는 한번도 없어... 키키,키키 나도 여친 아니었으면 볼 일 없었을 거 같아
16,(지시) 질문하기,1,여친이 제파? 가자고 했던 거야?,어 키키 가보고 싶다고 해서 갔지
18,(단언) 주장하기,1,수성못 갔을 때 수달 봤었는데 너무 귀엽더라 ㅋ,수성못에 수달도 있나?
20,(단언) 주장하기,1,응 키키 한번 봤었다 물에서 나왔다가 쏙 들어가더랑,키키 난 수성못 갔을 때 한 번도 못 봤는데


전처리

In [11]:
# df = df.drop(columns=['Unnamed: 3'])

# df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [260]:
df_pet_s3 = df_pet_s3.drop(columns=['대화나누기'])

df_pet_s3.head()

,의도,입력,출력
0,(지시) 질문하기,이구아나나 파충류 키워본 적 있나?,아니 무서워서 어떻게 키워…
2,(지시) 질문하기,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서
4,(지시) 질문하기,어디 동물원 갔었엉?,제파라고 제주도에 있어 하하
6,(지시) 질문하기,거기 파충류 막 만지고 할 수 있어? 키키,아 키키 체험하는 곳이라서 만질 수 있다
9,(지시) 질문하기,이구아나 만졌을 때 느낌 어땠어?,거칠거칠한 느낌? 무서웠다


In [261]:
df_pet_s3.isnull().sum()

의도    32
입력     0
출력     0
dtype: int64

In [12]:
# df = df[~df['챗봇'].isna()]

# df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [226]:
# df.loc[0, '유저']

In [14]:
# model.encode(df.loc[0, '유저'])

array([-4.80605900e-01, -2.94869393e-01,  4.37900096e-01, -6.40137792e-01,
        3.28670628e-02, -3.42647493e-01, -5.47480583e-02,  1.73052475e-02,
       -4.08220917e-01, -5.06034255e-01, -1.68733329e-01, -3.98677140e-01,
       -1.24776825e-01, -9.71540660e-02, -1.65286273e-01,  5.72626200e-03,
        6.13689683e-02, -1.91312313e-01,  2.53917158e-01, -5.85019648e-01,
       -2.84425855e-01, -2.32035190e-01, -3.27080786e-01,  6.72973171e-02,
       -1.65573183e-05, -4.72336203e-01, -3.60022068e-01,  2.91879892e-01,
       -6.63861394e-01, -3.10574591e-01,  5.79525173e-01, -3.11722696e-01,
        1.47700924e-02, -2.12172821e-01,  2.22057775e-01, -1.73829064e-01,
       -3.78458530e-01, -4.20398176e-01, -2.38218799e-01,  6.38705641e-02,
       -1.15304112e-01, -2.44563922e-01, -5.00228465e-01,  1.68355435e-01,
       -6.58360064e-01, -8.91942084e-01, -6.26957059e-01, -3.21965277e-01,
       -7.05359042e-01,  3.71447295e-01, -5.45803249e-01,  7.76302293e-02,
        1.09864689e-01,  

In [185]:
model.encode(df_pet_s3.loc[0, '입력'])

array([-6.67561412e-01, -8.07261467e-01,  2.09884375e-01, -1.74596354e-01,
       -1.99549869e-01, -1.37862995e-01,  5.64490736e-01,  3.52973014e-01,
       -2.79791713e-01, -1.27956882e-01,  9.65613797e-02,  4.61558372e-01,
       -4.16496992e-02, -2.81037331e-01, -2.83905834e-01, -2.71216303e-01,
        2.12736368e-01, -8.15320536e-02,  5.24000943e-01, -1.29906762e+00,
       -2.46748909e-01, -2.48723656e-01,  1.40673831e-01, -1.27187625e-01,
       -5.60404956e-01,  4.83852863e-01, -4.15023625e-01, -1.19413599e-01,
        5.39478540e-01,  6.91743553e-01,  2.25633383e-02,  6.21081926e-02,
       -2.60848075e-01,  5.26181579e-01,  7.21419305e-02, -4.15729463e-01,
        2.26031944e-01, -4.83054787e-01, -4.41329032e-01, -2.60057282e-02,
       -5.96620217e-02, -1.77799180e-01,  4.63493094e-02,  1.31032825e-01,
       -7.56557584e-02, -3.33543152e-01,  8.79133344e-01, -4.38649595e-01,
        3.05891931e-01, -3.31415981e-01, -3.19116175e-01, -3.91250364e-02,
       -6.88868910e-02,  

유저 대화내용 인코딩

In [15]:
# df['embedding'] = pd.Series([[]] * len(df))

# df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

# df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806059, -0.2948694, 0.4379001, -0.6401378..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561571, -0.14506254, 0.2949032, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268236, 1.0945568, 0.10579172,..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679057, 0.46520707, 0.52850693, -0.507604..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277727, -0.37413934, 0.040532183, -0.862..."


In [262]:
df_pet_s3['embedding'] = pd.Series([[]] * len(df_pet_s3))

df_pet_s3['embedding'] = df_pet_s3['입력'].map(lambda x: list(model.encode(x)))

df_pet_s3.head()

In [ ]:
df_pet_s3.reset_index(drop=True, inplace=True)

In [ ]:
df_pet_s3.to_csv('df_pet_s3.csv', index=False)

간단한 챗봇

In [161]:
# text = '요즘 머리가 아프고 너무 힘들어'

# embedding = model.encode(text)

In [18]:
# df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

# df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806059, -0.2948694, 0.4379001, -0.6401378...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561571, -0.14506254, 0.2949032, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652, -0.081268236, 1.0945568, 0.10579172,...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679057, 0.46520707, 0.52850693, -0.507604...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277727, -0.37413934, 0.040532183, -0.862...",0.315118


In [ ]:
combined_string = ' '.join(df_pet_s3['출력'])

embedding = model.encode(combined_string)

In [ ]:
embedding

array([-2.21995920e-01, -7.45399892e-01, -3.09407800e-01, -3.70294839e-01,
        1.75760001e-01, -2.08056763e-01,  4.44619328e-01,  1.89782292e-01,
        2.23872006e-01,  8.90279561e-03,  3.27422559e-01, -3.76071900e-01,
       -1.44796371e-01, -3.54809135e-01, -4.47848260e-01, -3.40445071e-01,
        4.10340726e-01,  1.57935858e-01,  3.09727848e-01, -1.29450321e+00,
        7.91049823e-02, -1.84021205e-01,  1.00223228e-01, -1.61767423e-01,
        1.36328399e-01,  7.53836557e-02, -3.55705291e-01, -2.19020545e-02,
        4.85864133e-01,  2.19090432e-01,  1.54126182e-01,  6.02567047e-02,
        2.36612916e-01,  3.70393872e-01,  1.96220219e-01,  9.03563388e-03,
       -2.10025787e-01, -9.94513214e-01, -3.65719348e-01,  4.00857151e-01,
        1.61925614e-01, -1.50970146e-01, -1.72482014e-01, -3.37271243e-02,
       -2.74675816e-01, -1.50475159e-01,  5.12199044e-01,  2.71114595e-02,
       -1.91290945e-01, -2.52876163e-01, -4.41471040e-01,  1.33969218e-01,
       -1.60627007e-01, -

In [ ]:
df_pet_s3['distance'] = df_pet_s3['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df_pet_s3.head()

,의도,입력,출력,embedding,distance
0,(지시) 질문하기,이구아나나 파충류 키워본 적 있나?,아니 무서워서 어떻게 키워…,"[-0.6675614, -0.80726147, 0.20988438, -0.17459...",0.618206
1,(지시) 질문하기,키키 맞아 나도 무서워 파충류는 새끼도 징그럽다 ㅠㅠ,며칠 전에 제주도 가서 이구아나 만져보긴 했다 ㅜㅜ 여친이 자꾸 만져보라고 해서,"[-0.29378846, -0.442848, -0.3631586, -0.268306...",0.499411
2,(지시) 질문하기,어디 동물원 갔었엉?,제파라고 제주도에 있어 하하,"[-0.031901617, -0.9987527, 0.19465397, -0.2703...",0.459000
3,(지시) 질문하기,거기 파충류 막 만지고 할 수 있어? 키키,아 키키 체험하는 곳이라서 만질 수 있다,"[-0.53480875, -0.16112292, 0.3402396, -0.63357...",0.520052
4,(지시) 질문하기,이구아나 만졌을 때 느낌 어땠어?,거칠거칠한 느낌? 무서웠다,"[-0.3376092, -0.61579233, 0.03611116, 0.020250...",0.640568


In [ ]:
answer = df_pet_s3.loc[df_pet_s3['distance'].idxmax()]

print('구분', answer['의도'])
print('유사한 질문', answer['입력'])
print('챗봇 답변', answer['출력'])
print('유사도', answer['distance'])

구분 (지시) 질문하기
유사한 질문 이구아나 만졌을 때 느낌 어땠어?
챗봇 답변 거칠거칠한 느낌? 무서웠다
유사도 0.6405675411224365
